In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install datasets


In [ ]:
import pandas as pd

# 파일 경로 (Google Drive 경로)
csv_file_path = '/content/drive/MyDrive/data_file/train.csv'

# 데이터 불러오기
data = pd.read_csv(csv_file_path)

# 데이터 확인
print(data.head())


            ID                                              input  \
0  TRAIN_00000  별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈...   
1  TRAIN_00001                             잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ   
2  TRAIN_00002                                    절테 간면 않 된는 굣 멥몫   
3  TRAIN_00003  야... 칵컥 좋꾜 부됴 뼝 뚫렷썹 신원햐쥠만 닮패 넴센 밌쪄벅림. 샥퀘 핥류만 묵...   
4  TRAIN_00004  집윈 축쳐눌료 딴너왓눈뎁 카셩뷔 좋곱 칼쿰한네올. 쩌럼한뒈 뮬콰 욺료토 잊쿄 빻토 ...   

                                              output  
0  별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...  
1                             잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ  
2                                    절대 가면 안 되는 곳 메모  
3  아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...  
4  지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...  


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# 모델 이름 설정
model_name = "google/byt5-base"

# 모델 및 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

In [ ]:
# 데이터셋 나누기 (Hugging Face Datasets의 train_test_split 사용)
dataset = tokenized_datasets  # 전체 데이터셋
split = dataset.train_test_split(test_size=0.1, seed=42)

# 나눠진 데이터셋
dataset1 = split['train']
dataset2 = split['test']



In [ ]:
import torch
from transformers import TrainerCallback, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoConfig

model_name = "google/byt5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dropout 설정 강제 적용
model.config.dropout_rate = 0.4
model.config.attention_probs_dropout_prob = 0.4
model.config.activation_dropout = 0.4
print(model.config)



class CustomLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            step = state.global_step
            epoch = state.epoch

        
            lr = float(logs.get("learning_rate", 0.0))  # 기본값 0.0
            loss = float(logs.get("loss", 0.0))  # 기본값 0.0
            eval_loss = float(logs.get("eval_loss", 0.0)) if "eval_loss" in logs else "N/A"  # eval_loss는 N/A 허용
            grad_norm = float(logs.get("grad_norm", 0.0))  # 기본값 0.0

            print(f"Step {step} | Epoch {epoch:.2f} | Loss: {loss:.4f} | Eval Loss: {eval_loss} | LR: {lr:.6f} | Grad Norm: {grad_norm:.4f}")




new_output_dir = "/content/drive/MyDrive/byt5base_dropout" 


training_args = TrainingArguments(
    output_dir=new_output_dir,  
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=7,
    weight_decay=0.02,
    logging_steps=100,
    log_level="info",
    logging_dir="./logs",
    bf16=True, 
    gradient_accumulation_steps=2, 
    max_grad_norm=1.0,  
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
    callbacks=[CustomLoggerCallback()]
)


trainer.train()
trainer.save_model(new_output_dir) 
trainer.state.save_to_json(os.path.join(new_output_dir, "trainer_state.json"))


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--byt5-base/snapshots/92d8c008d55cf7c254915bac165171dfe6c20c44/config.json
Model config T5Config {
  "_name_or_path": "google/byt5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.48.3",
  "use_cache": tru

T5Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/byt5-base",
  "activation_dropout": 0.4,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "attention_probs_dropout_prob": 0.4,
  "classifier_dropout": 0.0,
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.4,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.48.3",
  "use_cache": true,
  "vocab_size": 384
}



Safetensors PR exists
Using auto half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, ID, input. If output, ID, input are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,136
  Num Epochs = 7
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 17,738
  Number of trainable parameters = 581,653,248
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,0.086700,0.055986
2,0.053000,0.035725
3,0.039000,0.027957
4,0.032000,0.023980
5,0.028100,0.022156
6,0.024000,0.021350
7,0.022800,0.020894


Step 100 | Epoch 0.04 | Loss: 11.7839 | Eval Loss: N/A | LR: 0.000050 | Grad Norm: 16.6944
Step 200 | Epoch 0.08 | Loss: 3.1686 | Eval Loss: N/A | LR: 0.000049 | Grad Norm: 65.3220
Step 300 | Epoch 0.12 | Loss: 1.1969 | Eval Loss: N/A | LR: 0.000049 | Grad Norm: 4.2999
Step 400 | Epoch 0.16 | Loss: 0.5611 | Eval Loss: N/A | LR: 0.000049 | Grad Norm: 0.7582
Step 500 | Epoch 0.20 | Loss: 0.3337 | Eval Loss: N/A | LR: 0.000049 | Grad Norm: 0.6891
Step 600 | Epoch 0.24 | Loss: 0.3291 | Eval Loss: N/A | LR: 0.000048 | Grad Norm: 0.6349
Step 700 | Epoch 0.28 | Loss: 0.2573 | Eval Loss: N/A | LR: 0.000048 | Grad Norm: 0.3999
Step 800 | Epoch 0.32 | Loss: 0.2117 | Eval Loss: N/A | LR: 0.000048 | Grad Norm: 0.6992
Step 900 | Epoch 0.36 | Loss: 0.1933 | Eval Loss: N/A | LR: 0.000047 | Grad Norm: 0.5597
Step 1000 | Epoch 0.39 | Loss: 0.1793 | Eval Loss: N/A | LR: 0.000047 | Grad Norm: 0.5993
Step 1100 | Epoch 0.43 | Loss: 0.1790 | Eval Loss: N/A | LR: 0.000047 | Grad Norm: 0.1223
Step 1200 | Epoc

...

Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout/checkpoint-17738
Configuration saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, ID, input. If output, ID, input are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1127
  Batch size = 4


Step 17738 | Epoch 7.00 | Loss: 0.0000 | Eval Loss: 0.020893584936857224 | LR: 0.000000 | Grad Norm: 0.0000


Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout/checkpoint-17738
Configuration saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout/checkpoint-17738/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)




Step 17738 | Epoch 7.00 | Loss: 0.0000 | Eval Loss: N/A | LR: 0.000000 | Grad Norm: 0.0000


Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout
Configuration saved in /content/drive/MyDrive/byt5base_dropout/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout/model.safetensors


dropout적용시킨후 lr도 올리기

In [ ]:
import torch
from transformers import TrainerCallback, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoConfig

model_name = "google/byt5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Dropout 설정 강제 적용
model.config.dropout_rate = 0.4
model.config.attention_probs_dropout_prob = 0.4
model.config.activation_dropout = 0.4
print(model.config)



class CustomLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            step = state.global_step
            epoch = state.epoch
            lr = float(logs.get("learning_rate", 0.0))  # 기본값 0.0
            loss = float(logs.get("loss", 0.0))  # 기본값 0.0
            eval_loss = float(logs.get("eval_loss", 0.0)) if "eval_loss" in logs else "N/A"  # eval_loss는 N/A 허용
            grad_norm = float(logs.get("grad_norm", 0.0))  # 기본값 0.0

            print(f"Step {step} | Epoch {epoch:.2f} | Loss: {loss:.4f} | Eval Loss: {eval_loss} | LR: {lr:.6f} | Grad Norm: {grad_norm:.4f}")




new_output_dir = "/content/drive/MyDrive/byt5base_dropout3" \


training_args = TrainingArguments(
    output_dir=new_output_dir, \
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=7,
    weight_decay=0.02,
    logging_steps=100,
    log_level="info",
    logging_dir="./logs",
    bf16=True, 
    gradient_accumulation_steps=2,  
    max_grad_norm=1.0,  
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
    callbacks=[CustomLoggerCallback()]
)


trainer.train()
trainer.save_model(new_output_dir)  # ✅ 새로운 폴더에 저장
trainer.state.save_to_json(os.path.join(new_output_dir, "trainer_state.json"))


T5Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/byt5-base",
  "activation_dropout": 0.4,
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "attention_probs_dropout_prob": 0.4,
  "classifier_dropout": 0.0,
  "d_ff": 3968,
  "d_kv": 64,
  "d_model": 1536,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.4,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 12,
  "num_layers": 18,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.48.3",
  "use_cache": true,
  "vocab_size": 384
}



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Safetensors PR exists
Using auto half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input, ID, output. If input, ID, output are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,136
  Num Epochs = 7
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 17,738
  Number of trainable parameters = 581,653,248
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently s

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jseom818 (jseom818-yonsei-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


...

Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input, ID, output. If input, ID, output are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1127
  Batch size = 4


Step 17738 | Epoch 7.00 | Loss: 0.0000 | Eval Loss: 0.016352545469999313 | LR: 0.000000 | Grad Norm: 0.0000


Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout3/checkpoint-17738/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)




Step 17738 | Epoch 7.00 | Loss: 0.0000 | Eval Loss: N/A | LR: 0.000000 | Grad Norm: 0.0000


Saving model checkpoint to /content/drive/MyDrive/byt5base_dropout3
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/config.json
Configuration saved in /content/drive/MyDrive/byt5base_dropout3/generation_config.json
Model weights saved in /content/drive/MyDrive/byt5base_dropout3/model.safetensors


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, ByT5Tokenizer
import pandas as pd
import os

# GPU 최적화 설정
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 데이터 로드
test_data_path = "/content/drive/MyDrive/data_file/test.csv"
if not os.path.exists(test_data_path):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {test_data_path}")

df_test = pd.read_csv(test_data_path)
if "input" not in df_test.columns or "ID" not in df_test.columns:
    raise ValueError("`test.csv` 파일에 `input` 또는 `ID` 컬럼이 없습니다.")

# 모델 로드 및 BF16 설정
model_path = "/content/drive/MyDrive/byt5base_dropout3/checkpoint-12670"
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device).to(torch.bfloat16)
tokenizer = ByT5Tokenizer.from_pretrained("google/byt5-base")

# 모델 연산 최적화
model = torch.compile(model)
model.eval()

# 배치 크기 조정
batch_size = 8
total_size = len(df_test)

# 결과 저장 리스트
predictions = []

# 배치 단위로 처리 (Beam Search 적용)
for i in range(0, total_size, batch_size):
    batch_texts = df_test["input"].iloc[i: i + batch_size].tolist()

    inputs = tokenizer(
        batch_texts, return_tensors="pt", padding="longest", truncation=True, max_length=2096
    ).to(device)

    # BF16 적용 & Beam Search로 글로벌 최적해답 탐색
    with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        outputs = model.generate(
            **inputs,
            num_beams=5,                
            early_stopping=True,        
            length_penalty=1.0,         
            max_length=len(inputs["input_ids"][0])  # 입력과 동일한 길이 유지
        )

    # 결과 디코딩 및 저장
    batch_results = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    predictions.extend(batch_results)

    # 진행 상황 출력
    progress = (i + batch_size) / total_size * 100
    if i % (batch_size * 10) == 0 or i + batch_size >= total_size:
        print(f"🚀 Progress: {progress:.2f}% completed.")

# 예측 결과 저장
df_test["output"] = predictions
submission_path = "/content/drive/MyDrive/submission11.csv"
df_test[["ID", "output"]].to_csv(submission_path, index=False, encoding="utf-8")

print(f"✅ Submission file saved successfully: {submission_path}")


🚀 Using device: cuda
🚀 Progress: 0.47% completed.
🚀 Progress: 5.21% completed.
🚀 Progress: 9.95% completed.
🚀 Progress: 14.68% completed.
🚀 Progress: 19.42% completed.
🚀 Progress: 24.16% completed.
🚀 Progress: 28.89% completed.
🚀 Progress: 33.63% completed.
🚀 Progress: 38.37% completed.
🚀 Progress: 43.10% completed.
🚀 Progress: 47.84% completed.
🚀 Progress: 52.58% completed.
🚀 Progress: 57.31% completed.
🚀 Progress: 62.05% completed.
🚀 Progress: 66.79% completed.
🚀 Progress: 71.52% completed.
🚀 Progress: 76.26% completed.
🚀 Progress: 80.99% completed.
🚀 Progress: 85.73% completed.
🚀 Progress: 90.47% completed.
🚀 Progress: 95.20% completed.
🚀 Progress: 99.94% completed.
🚀 Progress: 100.41% completed.
✅ Submission file saved successfully: /content/drive/MyDrive/submission11.csv
